# Robustness and Plasticity in Regulatory Networks

* [Introduction](./RPRN-Introduction.ipynb)
* [BoolNet](./RPRN-BoolNet.ipynb)
* [Functions](./RPRN-Functions.ipynb)
* [Updating](./RPRN-Updating.ipynb)
* [States and trajectories](#States-and-Trajectories)
    * [Transient perturbations](#Transient-perturbations)
    * [Stochastic noise](#Stochastic-noise)
* [Appendix](./RPRN-Appendix.ipynb)

# States and Trajectories

Until now we have discused perturbations that occur in the functions or the evaluation of the regulatory network. However, once the attractors have been reached, they are subjected to other perturbations in the state of the system. This perturbations do not alter the wiring and updating of the network, but its state. This perturbations are transient, as the functions remain the same, it is only the state that is perturbed for a certain time, affecting its trajectory.

Biologicaly, this is equivalent to signals of the environment or intrinsic process of the cell that change the expression of an element for a certain time. For example, most drugs change the micro-environment of the organism while they are ingested, but once the treatment ends the perturbation ends. An other example of transient perturbations in biological systems is stochastic noise.

In this tutorial we will use BoolNet functions to study the effect of transiently perturbing the state of the regulatory network. We will also use some of the BoolNet-extensions funtions.

In [1]:
#Uncomment next line if you haven't installed BoolNet
#install.packages("BoolNet", repos='http://cran.us.r-project.org')
library(BoolNet)
library(BoolNetPerturb)

## Transient perturbations

Once a cell has differentiated, it is subject to a changing intrinsic and extrinsic environment. Intrinsic processes or the signals in the environment, can temporarly change the expression of some of their nodes.

A biological example is drugs. While drugs are ingested they alter the micro-environment in specific ways, but once the treatment ends the signal ends. Understanding what happens not only during the treatment, but after, is fundamental for medicine.

To study the effect We will use the labeled attractors from the Th17/iTreg network.

In [2]:
net <- loadNetwork("minTh17iTreg.txt")
attr <- getAttractors(net)
attr.df <- attractor2dataframe(attr) 

We will also need the environments and labeling rules.

In [3]:
environments <- list(
    label = c("All","pro-Th0", "pro-Th17", "pro-iTreg", "pro-IL10"),
    nodes = c(NA, rep(list(c("IL2e", "IL21e", "TGFBe", "IL10e")), 4)),
    values = list(NA, c(0,0,0,0), c(0,1,1,0), c(1,0,1,0), c(0,0,0,1))
)
#environments

In [4]:
labels.rules <- data.frame(
    labels = c('Th0', 'Th17', 'Treg', 'IL10+', 'TGFB+', 'RORGT+'),
    rules  = c('!(RORGT | FOXP3 | TGFB | IL10)', 
            'RORGT & STAT3', 
            'FOXP3 & TGFB', 
            'IL10', 
            'TGFB & ! (RORGT | FOXP3)', 
            'RORGT & ! STAT3' ),
    stringsAsFactors = FALSE
)
#labels.rules

Using this we can label the attractors.

In [5]:
labels <- labelAttractors(attr, net$genes, labels.rules$labels, labels.rules$rules)
attr.df$label <- sapply(labels, function(l) paste(as.character(l), collapse='/')  )
    
attr.df

,involvedStates,basinSize,label
1,0,27,Th0
2,1,2,Th0
3,4,14,Th0
4,16,13,TGFB+
5,48,8,IL10+TGFB+
6,65,35,Th0
7,68,8,Th0
8,89,13,Treg
9,112,3,IL10+TGFB+
10,121,1,TregIL10+


Now, lets take Th0 attractor 0 and expose it to a Th17 environment for one time step.

In [17]:
state <- 0
label <- labelState(dec2binState(state, net$genes), 
                    net$genes, labels.rules$labels, labels.rules$rules )
new.attr <- perturbPathToAttractor(dec2binState(state, net$genes), net, 
                c('IL2e', 'IL21e', 'TGFBe', 'IL10e'), c(0,1,1,0),
                time=1)

new.state <- new.attr$attractors[[1]]$involvedStates
new.label <- labelState(dec2binState(new.state, net$genes), 
                        net$genes, labels.rules$labels, labels.rules$rules )

new.state
new.label

4


[1] "Th0"

As we can see, this time, the resulting attractor is different, but the resulting label is the same. Lets see the trajectory this perturbation followed.

In [15]:
state <- 0
new.traj <- perturbPathToAttractor(dec2binState(state, net$genes), net, 
                c('IL2e', 'IL21e', 'TGFBe', 'IL10e'), c(0,1,1,0),
                time=1, returnTrajectory = TRUE)
new.traj

,IL2,RORGT,STAT3,FOXP3,TGFB,IL10,IL2e,IL21e,TGFBe,IL10e
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0
3,0,0,1,0,1,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0


A question we can ask ourselves is if an attractor is more sensible to perturbations in a particular node, and what transitions do this perturbations cause.

Lets take the Th0 __0__ attractor again and perturb one by one all its nodes for one time step. We will save this information in a data.frame

In [16]:
state <- 0
state <- dec2binState(state, net$genes)

# perturb all nodes for time = 1
perturbations <- lapply(names(state), function(node) {
    new.state <- perturbPathToAttractor(state, net, 
                node, !state[[node]], time=1)
    new.state$attractors[[1]]$involvedStates
})
# label states
pertur.labels <- sapply(perturbations, function(state) {
    state <- dec2binState(state, net$genes)
    label <- labelState(state, net$genes, labels.rules$labels, labels.rules$rules )
})
# create dataframe
perturbations <- data.frame(perturb.node=names(state), states=unlist(perturbations), labels=pertur.labels)
perturbations

,perturb.node,states,labels
1,IL2,1,Th0
2,RORGT,4,Th0
3,STAT3,4,Th0
4,FOXP3,16,TGFB+
5,TGFB,16,TGFB+
6,IL10,0,Th0
7,IL2e,1,Th0
8,IL21e,4,Th0
9,TGFBe,16,TGFB+
10,IL10e,0,Th0


This table shows us all the possible one node perturbations for a state and their resulting attractors and labels. As we can see the attractor is very sensible to perturbations, but most of them end in the same cell-type.

## Stochastic noise

# Index

* [Introduction](./RPRN-Introduction.ipynb)
* [BoolNet](./RPRN-BoolNet.ipynb)
* [Functions](./RPRN-Functions.ipynb)
* [Updating](./RPRN-Updating.ipynb)
* States and trajectories
* [Appendix](./RPRN-Appendix.ipynb)